# Lab 2: Computation of Eigenvalues and Eigenvectors

## 1. Rayleigh Quotient Iteration

The Rayleigh quotient can be used in conjunction with the Shifted Inverse Power Method (sIPM). The sIPM converges to the eigenvector associated with the eigenvalue closest to the shift $s$, and convergence is faster when this distance is small. By updating the shift dynamically using the Rayleigh quotient, we obtain the **Rayleigh Quotient Iteration (RQI)**, which accelerates convergence significantly.

### Algorithm: Rayleigh Quotient Iteration (RQI)

Given:
- A matrix $A$
- An initial vector $x_0$
- Maximum number of iterations $k_{max}$
- Initial shift $s$

Steps:
1. Set $\lambda_0 = s$
2. Normalize the initial vector: $u_{0} = x_{0} /||x_{0}||$
3. For $j = 1, 2, 3, \dots, k_{max}$:
   - Solve $(A-\lambda_{j-1}I)x_j = u_{j-1}$
   - Normalize: $u_{j} = x_{j} /||x_{j}||$
   - Compute the Rayleigh quotient: $\lambda_{j} = u^T_{j} A u_{j}$

RQI converges **quadratically** for simple eigenvalues and **cubically** for symmetric matrices, meaning it requires very few iterations to reach machine precision.

---

## Questions



(a) Implement the RQI algorithm. Run your RQI implementation using the matrix for $k_{max}=100$ and $s=100$:  
```python
A = np.array([[25, -41, 10, -6], [-41, 68, -17, 10], [10, -17, 5, -3], [-6, 10, -3, 2]])
```
At each iteration, compute the condition number of  $(A - \lambda_j I)$ using `np.linalg.cond` and plot it on a **semilog-y scale**. What do you observe about the growth of the condition number? Why does the condition number increase significantly as the iteration progresses?  

(b) To prevent ill-conditioning, include the following stopping criterions. Run your modified code and check whether it prevents ill-conditioning.

- Check the variations: if $|\lambda_j - \lambda_{j-1}| < \text{tol}$, stop.

- Use a Relative Condition Number Threshold:
$$
\text{cond}(A - \lambda_j I) > \frac{1}{\text{machine epsilon}}
$$
```python
if np.linalg.cond(A - lam_new * np.eye(A.shape[0])) > 1 / np.finfo(float).eps:
    break  # Stop if nearly singular
```
This ensures the algorithm stops when the matrix is numerically unstable.

- Track the Condition Number Growth: Instead of stopping abruptly when the condition number crosses the threshold, check its growth rate. If it increases exponentially across multiple iterations, stop early:
```python
if len(cond_history) > 2 and cond_history[-1] > 10 * cond_history[-2]:
    break  # Stop if condition number increases too fast
```

- Use Residual-Based Stopping: An alternative approach is to stop when the **residual** becomes too large:  
$$
\| (A - \lambda_j I) x_j \| < \text{tol}
$$
This ensures we halt when the computed eigenvector no longer improves:
```python
residual = np.linalg.norm((A - lam_new * np.eye(A.shape[0])) @ x_new)
if residual < tol:
    break
```

(c) Compare the efficiency of sIPM and RQI when applied to a $N \times N$ random symmetric matrix. Use:

  ```python
  import numpy as np
  import matplotlib.pyplot as plt

  N_values = [4, 8, 16, 32, 64, 128, 256]
  iterations_sIPM = []
  iterations_RQI = []

  for N in N_values:
      A = np.random.rand(N, N)
      B = (A @ A.T) / 2  # symmetric matrix
      
      # Call your sIPM and RQI functions here to get iterations and store in the lists
      # Example:
      # iterations_sIPM.append(run_sIPM(B, ...))
      # iterations_RQI.append(run_RQI(B, ...))

  plt.figure()
  plt.semilogy(N_values, iterations_sIPM, label='sIPM')
  plt.semilogy(N_values, iterations_RQI, label='RQI')
  plt.xlabel('Matrix Size N')
  plt.ylabel('Number of Iterations')
  plt.legend()
  plt.show()
  ```

with

  - `k = 1e4` -- maximum number of iterations
  - `s = 100` -- initial shift
  - `x = np.random.rand(N, 1)` -- initial vector
  


(d) Compare the convergence of sIPM and RQI as a function of the number of iterations when computing the eigenvalues of a $24 \times 24$ random symmetric matrix. For that purpose, you need to modify both scripts so that:

  - The computed eigenvalue $\hat{\lambda}$ is stored in an array.
  - Use `eig` from `numpy.linalg` to obtain the reference eigenvalue $\lambda_e$.
  - Plot in the same figure the error $|\hat{\lambda} - \lambda_e|$ as a function of the number of iterations in a `semilogy` scale. Make sure to select the right reference eigenvalue when computing the error.

  Use:

  ```python
  from numpy.linalg import eig

  N = 24
  B = np.random.rand(N, N)
  B = (B @ B.T) / 2  # symmetric matrix
  x = np.random.rand(N, 1)  # initial vector
  k = 20  # maximum number of iterations
  s = 100  # initial shift
  
  reference_eigenvalues, _ = eig(B)

  computed_eigenvalues_sIPM = []
  computed_eigenvalues_RQI = []

  for iteration in range(k):
      # Run sIPM and RQI, storing the computed eigenvalue at each iteration
      # Example:
      # computed_eigenvalues_sIPM.append(run_sIPM(B, x, s, iteration))
      # computed_eigenvalues_RQI.append(run_RQI(B, x, s, iteration))
      pass
  
  error_sIPM = np.abs(np.array(computed_eigenvalues_sIPM) - reference_eigenvalues[0])  # Assuming using first eigenvalue
  error_RQI = np.abs(np.array(computed_eigenvalues_RQI) - reference_eigenvalues[0])

  plt.figure()
  plt.semilogy(range(k), error_sIPM, label='sIPM Error')
  plt.semilogy(range(k), error_RQI, label='RQI Error')
  plt.xlabel('Iterations')
  plt.ylabel('Error (log scale)')
  plt.legend()
  plt.show()
  ```

  - `B` -- matrix
  - `x = np.random.rand(N, 1)` -- initial vector
  - `k = 20` -- maximum number of iterations
  - `s = 100` -- initial shift
